# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
R. Zhang  ->  R. Zhang  |  ['R. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
N. Storm  ->  N. Storm  |  ['N. Storm']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
R. Hoppe  ->  R. Hoppe  |  ['R. Hoppe']
M. Habouzit  ->  M. Habouzit  |  ['M. Habouzit']


Arxiv has 60 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2401.13027


extracting tarball to tmp_2401.13027...

 done.
Retrieving document from  https://arxiv.org/e-print/2401.13167


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2401.13027/ms_arXiv.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'affilations' from 'tmp_2401.13027/affilations.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'authors' from 'tmp_2401.13027/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

extracting tarball to tmp_2401.13515...

 done.


M. Habouzit  ->  M. Habouzit  |  ['M. Habouzit']


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure dkl_dist
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure dkl_dist as tmp_2401.13515/./dkl_dist.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure dkl_vs_counts
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure dkl_vs_counts as tmp_2401.13515/./dkl_vs_counts.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-package

Found 189 bibliographic references in tmp_2401.13515/aanda.bbl.
syntax error in line 676: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.13450-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.13450) | **3D NLTE modelling of Y and Eu. Centre-to-limb variation and solar  abundances**  |
|| <mark>N. Storm</mark>, et al. -- incl., <mark>M. Bergemann</mark>, <mark>R. Hoppe</mark> |
|*Appeared on*| *2024-01-25*|
|*Comments*| *8 pages, 4 figures, accepted by A&A*|
|**Abstract**| Context. Abundances of s- and r-process elements in Sun-like stars constrain nucleosynthesis in extreme astrophysical events, such as compact binary mergers and explosions of highly magnetised rapidly rotating massive stars. Aims. We measure solar abundances of yttrium (Y) and europium (Eu) using 3D non-local thermal equilibrium (NLTE) models. We use the model to determine the abundance of Y, and also explore the model's ability to reproduce the solar centre-to-limb variation of its lines. In addition, we determine the Eu abundance using solar disc-centre and integrated flux spectra. Methods. We developed an NLTE model of Eu and updated our model of Y with collisional data from detailed quantum-mechanical calculations. We used the IAG spatially resolved high-resolution solar spectra to derive the solar abundances of Y across the solar disc and of Eu for integrated flux and at disc centre using a set of carefully selected lines and a 3D radiation-hydrodynamics model of the solar atmosphere. Results. We find 3D NLTE solar abundances of A(Y)$_{\textrm{3D NLTE}}$=$2.30 \pm 0.03_{\textrm{stat}} \pm 0.07_{\textrm{syst}}$ dex based on observations at all angles and A(Eu)$_{\textrm{3D NLTE}}$=$0.57 \pm 0.01_{\textrm{stat}} \pm 0.06_{\textrm{syst}}$ dex based on the integrated flux and disc-centre intensity. 3D NLTE modelling offers the most consistent abundances across the solar disc, and resolves the problem of severe systematic bias in Y and Eu abundances inherent to 1D LTE, 1D NLTE, and 3D LTE modelling. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.13515-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.13515) | **AGN X-ray luminosity function and absorption function in the Early  Universe ($3\leq z \leq 6$)**  |
|| E. Pouliasis, et al. -- incl., <mark>M. Habouzit</mark> |
|*Appeared on*| *2024-01-25*|
|*Comments*| *Accepted for publication in A&A. 25 pages, 15 figures (+4 in Appendix), 2 tables (+1 in Appendix)*|
|**Abstract**| The XLF of AGN offers a robust tool to study the evolution and the growth of SMBHs over cosmic time. Owing to the limited area probed by X-ray surveys, optical surveys are routinely used to probe the accretion in the high redshift Universe $z\geq 3$. However, optical surveys may be incomplete because they are strongly affected by dust redenning. In this work, we derive the XLF and its evolution at high redshifts using a large sample of AGNs selected in different fields with various areas and depths covering a wide range of luminosities. Additionally, we put the tightest yet constraints on the absorption function in this redshift regime. In particular, we use more than 600 soft X-ray selected high-z sources in the Chandra Deep fields, the Chandra COSMOS Legacy survey and the XMM-XXL northern field. We derive the X-ray spectral properties for all sources via spectral fitting, using a consistent technique and model. For modeling the parametric form of the XLF and the absorption function, we use a Bayesian methodology allowing us to correctly propagate the uncertainties for the observed X-ray properties of our sources and also the absorption effects. The evolution of XLF is in agreement with a pure density evolution model similar to what is witnessed at optical wavelengths, although a luminosity dependent density evolution model cannot be securely ruled out. A large fraction ($60\%)$ of our sources are absorbed by column densities of $\rm N_H \geq 10^{23} cm^{-2} $, while $17$\% of the sources are CTK. Our results favor a scenario where both the ISM of the host and the AGN torus contribute to the obscuration. The derived BHAD is in agreement with the simulations, if one takes into account that the X-ray AGN are hosted by massive galaxies, while it differs from the one derived using JWST data. The latter could be due to the differences in the AGN and host-galaxy properties. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.13027-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.13027) | **Nightside clouds and disequilibrium chemistry on the hot Jupiter  WASP-43b**  |
|| T. J. Bell, et al. -- incl., <mark>L. Kreidberg</mark>, <mark>E.-M. Ahrer</mark> |
|*Appeared on*| *2024-01-25*|
|*Comments*| *61 pages, 13 figures, 4 tables. This preprint has been submitted to and accepted in principle for publication in Nature Astronomy without significant changes*|
|**Abstract**| Hot Jupiters are among the best-studied exoplanets, but it is still poorly understood how their chemical composition and cloud properties vary with longitude. Theoretical models predict that clouds may condense on the nightside and that molecular abundances can be driven out of equilibrium by zonal winds. Here we report a phase-resolved emission spectrum of the hot Jupiter WASP-43b measured from 5-12 $\mu$m with JWST's Mid-Infrared Instrument (MIRI). The spectra reveal a large day-night temperature contrast (with average brightness temperatures of 1524$\pm$35 and 863$\pm$23 Kelvin, respectively) and evidence for water absorption at all orbital phases. Comparisons with three-dimensional atmospheric models show that both the phase curve shape and emission spectra strongly suggest the presence of nightside clouds which become optically thick to thermal emission at pressures greater than ~100 mbar. The dayside is consistent with a cloudless atmosphere above the mid-infrared photosphere. Contrary to expectations from equilibrium chemistry but consistent with disequilibrium kinetics models, methane is not detected on the nightside (2$\sigma$ upper limit of 1-6 parts per million, depending on model assumptions). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.13167-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.13167) | **A star-based method for precise flux calibration of the Chinese Space  Station Telescope (CSST) slitless spectroscopic survey**  |
|| L. Yang, et al. -- incl., <mark>R. Zhang</mark> |
|*Appeared on*| *2024-01-25*|
|*Comments*| *20 pages, 15 figures, accepted by ApJS*|
|**Abstract**| The upcoming Chinese Space Station Telescope (CSST) slitless spectroscopic survey poses a challenge of flux calibration, which requires a large number of flux-standard stars. In this work, we design an uncertainty-aware residual attention network, the UaRA-net, to derive the CSST SEDs with a resolution of R = 200 over the wavelength range of 2500-10000 \AA using LAMOST normalized spectra with a resolution of R = 2000 over the wavelength range of 4000-7000 \AA. With the special structure and training strategy, the proposed model can not only provide accurate predictions of SEDs but also their corresponding errors. The precision of the predicted SEDs depends on effective temperature (Teff), wavelength, and the LAMOST spectral signal-to-noise ratios (SNRs), particularly in the GU band. For stars with Teff = 6000 K, the typical SED precisions in the GU band are 4.2%, 2.1%, and 1.5% at SNR values of 20, 40, and 80, respectively. As Teff increases to 8000 K, the precision increases to 1.2%, 0.6%, and 0.5%, respectively. The precision is higher at redder wavelengths. In the GI band, the typical SED precisions for stars with Teff = 6000 K increase to 0.3%, 0.1%, and 0.1% at SNR values of 20, 40, and 80, respectively. We further verify our model using the empirical spectra of the MILES and find good performance. The proposed method will open up new possibilities for optimal utilization of slitless spectra of the CSST and other surveys. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.13319-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.13319) | **Two high-amplitude $δ$ Scuti-$γ$ Doradus hybrids constrained  by the radial fundamental p and equally spaced g modes**  |
|| X. Chen, <mark>X. Zhang</mark>, Y. Li, J. Su |
|*Appeared on*| *2024-01-25*|
|*Comments*| *16 pages, 6 figures, 5 tables, accepted for publication in The Astrophysical Journal*|
|**Abstract**| Based on 2-minute cadence TESS data, we investigate pulsations of TIC 65138566 and TIC 139729335 and discover them to be two new HADS stars with equally spaced g modes. We recognize the radial fundamental mode $f_1$ = 18.3334 c/d and the first overtone $f_3$ = 23.6429 c/d for TIC 65138566, and identify the highest peak $f_1$ = 19.0955 c/d as the radial fundamental mode for TIC 139729335. For g modes, both stars display a regular period spacing of 2413 s. Through detailed seismological analysis, we deduce that these period spacing patterns correspond to modes with $\ell$ = 1. Moreover, our analysis reveals that with the increase in masses and metallicities, the star should display a higher degree of evolution to match a specific period spacing $\Pi_0$. Conversely, the star should have a lower extent of evolution to match the radial fundamental mode. These two contradictory behaviors allow us to precisely obtain stellar physical parameters. TIC 65138566 and TIC 139729335 are determined to be two main sequence stars that have almost the same range of masses and metallicities, with $M$ = 1.36 $\pm$ 0.06 $M_{\odot}$ and $Z$ = 0.005 $\pm$ 0.002. The hydrogen abundance in the core of TIC 65138566 is estimated to be about 0.28, while TIC 139729335 has a slightly higher value of around 0.31. Finally, we suggest that the HADS $\delta$ Scuti-$\gamma$ Doradus star TIC 308396022 is a main sequence star with $M$ = 1.54 $\pm$ 0.08 $M_{\odot}$, $Z$ = 0.007 $\pm$ 0.001, and $X_{\rm c}$ = 0.18 $\pm$ 0.02. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2401.13450.md
    + _build/html/tmp_2401.13450/./figures/plot_example_yttrium_lines_diff_mu_0_v2_arx.png
    + _build/html/tmp_2401.13450/./figures/plot_example_yttrium_lines_diff_mu_12_v2_arx.png
    + _build/html/tmp_2401.13450/./figures/y_abund_10_comb.png
exported in  _build/html/2401.13515.md
    + _build/html/tmp_2401.13515/./plot_xlf_all_z_ldde_pde2ssss2024.png
    + _build/html/tmp_2401.13515/./plot_bhadxVOL_SEP2023sB82024.png
    + _build/html/tmp_2401.13515/./FIG1_plot_area_curves.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\logg}{\log g}$
$\newcommand{\vmic}{\xi_{\rm t}}$
$\newcommand{\Vmic}{\xi_{\rm t}}$
$\newcommand{\vmac}{V_{\rm mac}}$
$\newcommand{\EW}{W_{\lambda}}$
$\newcommand{\mA}{{\rm mÅ}}$
$\newcommand{\Elow}{E_{\rm low}}$
$\newcommand{\Eup}{E_{\rm up}}$
$\newcommand{\SH}{S\!_{\rm H}}$
$\newcommand{\Eu}[5]{\mbox{#1 ^#2{\rm #3}^{{\rm #4}}_{\rm #5}}}$
$\newcommand{\Y}[5]{\mbox{#1 ^#2{\rm #3}^{{\rm #4}}_{\rm #5}}}$
$\newcommand{\footnoterule}$
$\newcommand{\footnoterule}$
$\newcommand{\footnoterule}$</div>



<div id="title">

# 3D NLTE modelling of Y and Eu

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2401.13450-b31b1b.svg)](https://arxiv.org/abs/2401.13450)<mark>Appeared on: 2024-01-25</mark> -  _8 pages, 4 figures, accepted by A&A_

</div>
<div id="authors">

<mark>N. Storm</mark>, et al. -- incl., <mark>M. Bergemann</mark>, <mark>R. Hoppe</mark>

</div>
<div id="abstract">

**Abstract:** Abundances of s- and r-process elements in Sun-like stars constrain nucleosynthesis in extreme astrophysical events, such as compact binary mergers and explosions of highly magnetised rapidly rotating massive stars. We measure solar abundances of yttrium (Y) and europium (Eu) using 3D non-local thermal equilibrium (NLTE) models. We use the model to determine the abundance of Y, and also explore the model's ability to reproduce the solar centre-to-limb variation of its lines. In addition, we determine the Eu abundance using solar disc-centre and integrated flux spectra. We developed an NLTE model of Eu and updated our model of Y with collisional data from detailed quantum-mechanical calculations. We used the IAG spatially resolved  high-resolutionsolar spectra to derive the solar abundances of Y across the solar disc and of Eu for integrated flux and at disc ${centre}$ using a set of carefully selected lines and a 3D radiation-hydrodynamics model of the solar atmosphere. We find 3D NLTE solar abundances of ${A(Y)$_{\textrm{3D NLTE}} $ = $2.30 \pm 0.03_{\textrm{stat}} \pm 0.07_{\textrm{syst}}$}$ dex based on observations at all angles and ${A(Eu) = $0.57 \pm 0.01_{\textrm{stat}} \pm 0.06_{\textrm{syst}}$}$ dex based on the integrated flux and disc-centre intensity. ${3D}$ NLTE modelling offers the most consistent abundances across the solar disc, and resolves the problem of severe systematic bias in Y and Eu abundances inherent to 1D LTE, 1D NLTE, and 3D LTE modelling.

</div>

<div id="div_fig1">

<img src="tmp_2401.13450/./figures/plot_example_yttrium_lines_diff_mu_0_v2_arx.png" alt="Fig1" width="100%"/>

**Figure 1. -** {Comparison of the observed Y II line profiles from the solar disc-centre ($\mu = 1.00$) intensity atlas (black) with the 3D NLTE (blue) and 1D LTE (red) model profiles with the instrumental broadening and a constant microturbulence of $\vmic = 1$ km/s for 1D models. A(Y) = 2.3 dex is used for all models for clear comparison.} (*fig:y_lines_examples_1.0*)

</div>
<div id="div_fig2">

<img src="tmp_2401.13450/./figures/plot_example_yttrium_lines_diff_mu_12_v2_arx.png" alt="Fig2" width="100%"/>

**Figure 2. -** Same as Fig. \ref{fig:y_lines_examples_1.0} but for the limb, $\mu = 0.20$. (*fig:y_lines_examples_0.2*)

</div>
<div id="div_fig3">

<img src="tmp_2401.13450/./figures/y_abund_10_comb.png" alt="Fig3" width="100%"/>

**Figure 3. -** {Abundances of Y determined from the IAG high-resolution solar observations taken at different viewing angles ($\mu$) using 1D LTE, 1D NLTE, 3D LTE, and 3D NLTE models. The results obtained using 1D LTE and 1D NLTE line formation models are shown in the left panel (a). The results obtained using 3D LTE and 3D NLTE models are shown in the right panel (b). The average solar A(Y) value and its standard deviation are provided in the figure inset for each model. See the main text for further details.} (*fig:y_fit_4883*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2401.13450"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\XMM}{\textit{XMM-Newton}}$
$\newcommand{\Chandra}{\textit{Chandra}}$
$\newcommand{\La}{\mathcal{L}}$
$\newcommand{\LX}{L_\mathrm{X}}$
$\newcommand{\mnh}{N_\mathrm{H}}$
$\newcommand{\nh}{N_\mathrm{H}}$
$\newcommand{\dlog}{\mathrm{dlog}}$
$\newcommand{\pden}{{p_\mathrm{den}}}$
$\newcommand{\fabs}{f_\mathrm{abs}}$
$\newcommand{\fctk}{f_{\mathrm{CTK},r}}$
$\newcommand{\Lbol}{L_\mathrm{bol}}$</div>



<div id="title">

# AGN X-ray luminosity function and absorption function in the Early Universe ($3\leq z \leq 6$)

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2401.13515-b31b1b.svg)](https://arxiv.org/abs/2401.13515)<mark>Appeared on: 2024-01-25</mark> -  _Accepted for publication in A&A. 25 pages, 15 figures (+4 in Appendix), 2 tables (+1 in Appendix)_

</div>
<div id="authors">

E. Pouliasis, et al. -- incl., <mark>M. Habouzit</mark>

</div>
<div id="abstract">

**Abstract:** The X-ray luminosity function (XLF) of active galactic nuclei (AGN) offers a robust tool to study the evolution and the growth of the super-massive black-hole population over cosmic time. Owing to the limited area probed by X-ray surveys, optical surveys are routinely used to probe the accretion in the high redshift Universe $z\geq 3$ . However, optical surveys may be incomplete because they are strongly affected by dust redenning.   In this work, we derive the XLF and its evolution at high redshifts ( $z\geq 3$ ) using a large sample of AGNs selected in different fields with various areas and depths covering a wide range of luminosities. Additionally, we put the tightest yet constraints on the absorption function in this redshift regime.   In particular, we use more than 600 soft X-ray selected ( $0.5-2$ keV) high-z sources in the Chandra Deep fields, the Chandra COSMOS Legacy survey and the XMM-XXL northern field. We derive the X-ray spectral properties for all sources via spectral fitting, using a consistent technique and model. For modeling the parametric form of the XLF and the absorption function, we use a Bayesian methodology allowing us to correctly propagate the uncertainties for the observed X-ray properties of our sources and also the absorption effects.   The evolution of XLF is in agreement with a pure density evolution model similar to what is witnessed at optical wavelengths, although a luminosity dependent density evolution model cannot be securely ruled out. A large fraction ( $\sim 60 \%)$ of our sources are absorbed by column densities of $\rm N_H \geq 10^{23} cm^{-2} $ , while $\sim 17$ \% of the sources are Compton-thick. Our results favor a scenario where both the interstellar medium of the host and the AGN torus contribute to the obscuration. The derived black hole accretion rate density is roughly in agreement with the large-scale cosmological hydro-dynamical simulations, if one takes into account the results that the X-ray AGN are hosted by massive galaxies, while it differs from the one derived using JWST data. The latter could be due to the differences in the AGN and host-galaxy properties.

</div>

<div id="div_fig1">

<img src="tmp_2401.13515/./plot_xlf_all_z_ldde_pde2ssss2024.png" alt="Fig14" width="100%"/>

**Figure 14. -** The best-fitting PDE model in several redshift bins computed by integrating the XLF over redshift and column density. The purple shaded regions represent the 68\% and 95\% confidence intervals of the model, while the purple data points indicate the binned luminosity function. We compare our results with the parametric forms of the XLF derived by previous X-ray studies \citep{Ueda2014,Vito2014,Georgakakis2015,Peca2023} in the column density interval of $\log N_{\rm H} = 20-24$(upper panels). Furthermore, we present our XLF integrating over $\log N_{\rm H} = 20-26$(lower panels) to the results of \citep{Buchner2015,Vito2018,Wolf2021,BarlowHall2023}. The XLF of \citet{Buchner2015} in the first two panels are derived in the redshift range $3.2 \leq z \leq 4.0$ and $4.0 \leq z \leq 7.0$, respectively. Our PDE model at the last panel is extrapolated at z=6.05. The brown dashed-dotted lines show the predicted XLF derived by \citet{Ananna2019}. (*xlf_full*)

</div>
<div id="div_fig2">

<img src="tmp_2401.13515/./plot_bhadxVOL_SEP2023sB82024.png" alt="Fig17" width="100%"/>

**Figure 17. -** Redshift evolution of the black hole accretion rate density. The shaded regions represent the 68\% and 99\% confidence intervals of the BHAD using the best-fitting PDE model. From left to right, we compare our results with previous X-ray studies \citep{Georgakakis2015,Ueda2014,Vito2018,Wolf2021,Peca2023}, with simulations \citep{Volonteri2016,Ni2022,Sijacki2015} and with the star-formation rate density scaled down by a factor of 3000 \citep{Madau2014, Harikane2022}. The black regions in the first two panels indicate the 99\% confidence interval of the BHAD adopting the bolometric correction by \citet{Lusso2012} so we can compare directly to the results of \citet{Vito2018}. The black points in the middle panel correspond to the _JWST_ results by \citet{Yang2023}. The brown dashed-dotted line shows the predicted XLF derived by \citet{Ananna2019}. (*bhad*)

</div>
<div id="div_fig3">

<img src="tmp_2401.13515/./FIG1_plot_area_curves.png" alt="Fig1" width="100%"/>

**Figure 1. -** X-ray sensitivity curves presented individually for the CDF-S/N, CCLS and XMM-XXL-N fields. The total area curve is shown with the orange solid line. (*area_curves*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2401.13515"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

361  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
